In [27]:
#用RNN判断影评背后的态度问题

import numpy as np
import tensorflow as tf
#读入数据
with open('reviews.txt', 'r') as f:
    reviews = f.read()
with open('labels.txt', 'r') as f:
    labels = f.read()
#预处理

from string import punctuation 
all_text = ''.join([c for c in reviews if c not in punctuation])  ##去掉标点符号

reviews = all_text.split('\n')  ##将每一条影评切分，成为list中的一个元素
all_text = ' '.join(reviews)   ###将每一条影评连接起来， 用空格（‘ ’）进行间隔
words = all_text.split() ##按照空格将所有影评 切割成单词

from collections import Counter
c = Counter(words)
d = sorted(c, key=c.get, reverse=True)
vocab_to_int = {word:ii for ii, word in enumerate(d, 0)} ##序号从1开始

reviews_ints = []
for e in reviews: ##e代表每条评论
    reviews_ints.append([vocab_to_int[word] for word in e.split()])  ##e是一段字符串，用split()进行切割
#print(reviews_ints[:3])


###将label变为1和0  positive=1， negative=0

# labels_dict ={'positive':1, 'negative':0} 
# all_labels = labels.split('\n')
# labelss = [labels_dict[v] for v in all_labels]
label = labels.split('\n')
labelss = np.array([1 if each == 'positive' else 0 for each in label])
#labelss = np.array(labelss)


####查看每条影评的长度，映射成为字典，key=影评长度， value=该长度出现的次数
from collections import Counter
reviews_lens = Counter(len(x) for x in reviews_ints)
print('Zero-length reviews:{}'.format(reviews_lens[0]))
print('Maxium review length:{}'.format(max(reviews_lens)))

# 发现影评中出现了一个长度为0的影评，我们需要删掉它

##找到不等于0的索引值
nozero_index = [i for i,v in enumerate(reviews_ints) if len(v)!=0]
#zero_index = [ii for ii,v in enumerate(reviews_ints) if len(v) == 0]
#len(nozero_index)

##将reviews_int 和 labels 中影评长度为0的对应值删除
reviews_ints = [reviews_ints[i] for i in nozero_index]
labelss = np.array([labelss[i] for i in nozero_index]) ##顺便将label转成数列形式


'''
创建一个包含我们传递给网络的数据的数组特性。 
数据应该来自review_ints，因为我们想将整数提供给网络。 
每行应该是200个元素长。 对于短于200字的评论，左边填充0。 
也就是说，如果评论是['最好'，'电影'，'永远']，[117,18,128]作为整数，
则该行将看起来像[0,0,0，...，0,117 ，18,128]。 
对于超过200的评论，使用前200个单词作为特征向量。
'''
seq_len = 200
features = np.zeros((len(reviews_ints),seq_len), dtype=int)

for i,row in enumerate(reviews_ints):
    '''
    每行进行一次循环，将reviews_ints从右边插入到feature矩阵中
    
    -len(row):  表示从倒数的row的长度 开始将0替换为row的值  如果len(row)的长度大于feature的列长度，也没关系，
    python的切片[:seq_len]  seq_len 大于len(row)也没问题   
    '''
    features[i, -len(row):] = np.array(row)[:seq_len]  
    
#训练数据集，验证数据集，测试数据集

##切分数据集
##将数据集切分成训练集和验证集  一般用0.8或者0.9
split_frac = 0.8 
split_index = int(len(features)*split_frac)
train_x, val_x = features[:split_index], features[split_index:]
train_y, val_y = labelss[:split_index], labelss[split_index:]
##再将验证集切分成验证集和测试集
test_index = int(len(val_x)*0.5)
val_x, test_x = val_x[:test_index], val_x[test_index:]
val_y, test_y = val_y[:test_index], val_y[test_index:]

print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(train_x.shape), 
      "\nValidation set: \t{}".format(val_x.shape),
      "\nTest set: \t\t{}".format(test_x.shape))



Zero-length reviews:1
Maxium review length:2514
			Feature Shapes:
Train set: 		(20000, 200) 
Validation set: 	(2500, 200) 
Test set: 		(2500, 200)


In [28]:
#RNN网络初始化

'''
建立图结构：

    lstm有四个网络层：3个sigma层和1个TNH层
        lstm_size:
            如果将lstm_size的值设为256
            则每一层都会有256个单元
            但是隐藏层的单元越多，效果就会越好，但是需要更多的运算时间
        lstm_layers:
            层数越多，拟合效果越好，
            当然也会出现过拟合的情况，我们可以加足够大的层数，
            通过其他方法，例如：dropout 正则化来防止过拟合
            这里从1开始调试
        batch_size:
            批数量：代表了每次放入网络的影评的数量
            在内存允许的情况下，越大越好，因为tensorflow对矩阵运算有优势，传入越大，一次迭代的时间就越少
        learning_rate:
            学习率
            
'''
lstm_size = 256
lstm_layers = 1
batch_size = 500
learning_rate = 0.001


n_words = len(vocab_to_int)

'''
定义输入标签和占位符

'''
graph = tf.Graph()
with graph.as_default():
    inputs_ = tf.placeholder(tf.int32, [None, None], name='inputs')
    labels_ = tf.placeholder(tf.int32, [None, None], name='labels')
    ##dropout的保留率
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    
    
'''
添加一个嵌入层。我们的词汇量有74000个单词。 进行one-hot编码是非常低效的。
创建一个嵌入层，并将该层用作查找表，而不是一个热编码。
可以使用word2vec来训练一个嵌入层，然后在这里加载它。
但是，创建一个新层并让网络学习权重是很好的。


创建矩阵tf.Variable。 
使用该嵌入矩阵通过tf.nn.embedding_lookup将嵌入的向量传递给LSTM单元。
这个函数采用嵌入矩阵和一个输入张量。 然后，它会返回嵌入式向量的另一个张量。
所以，如果嵌入层有200个单位，函数将返回一个大小为[batch_size，200]的张量。
'''
embed_size = 300 

with graph.as_default():
    #创建一个n_words*embed_size的矩阵，代表着每个词的词向量
    embedding = tf.Variable(tf.random_uniform((n_words, embed_size), -1,1))
    #将我们的影评转换成词向量表示
    embed = tf.nn.embedding_lookup(embedding, inputs_)
    
    
with graph.as_default():
    ##创建LSTM结构
    lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
    ##创建dropout结构(它包裹着lstm结构)
    drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
    # 叠加多个LSTM层，进行深度学习
    ##  [drop]*lstm_layers是tensorflow1.1之后的表达方式
    cell = tf.contrib.rnn.MultiRNNCell([drop]*lstm_layers)
    # 获得全零的初始状态
    initial_state = cell.zero_state(batch_size, tf.float32)

'''
创建了一个初始状态initial_state，传递给RNN。 
这是在连续时间步骤中在隐藏层之间传递的单元状态。 
tf.nn.dynamic_rnn负责我们大部分的工作。
返回每个时间步的outputs和隐藏层的final_state。
'''
with graph.as_default():
    outputs, final_state = tf.nn.dynamic_rnn(cell, embed,
                                             initial_state=initial_state)

##输出
'''
我们只关心最后的输出，
我们将用它作为我们的情绪预测。 
所以我们需要用输出[:, -1]来获取最后一个输出，
然后根据这个输出和labels_计算成本。
'''
with graph.as_default():
    ##加上全连接层 
    '''
    tf.contrib.layers.fully_connected（F, num_outputs,activation_fn）
    
    F ---[batch_size,images_pixels],tensor

    num_outputs --- numbers of outputs,[batch_size,num_outputs]

    activation_fn ---采用指定的非线性激励函数，默认不是None,如果不需要的话，要赋值None

    '''
    predictions = tf.contrib.layers.fully_connected(outputs[:, -1], 1, activation_fn=tf.sigmoid)  
    cost = tf.losses.mean_squared_error(labels_, predictions)
    
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
    

##验证精确度
with graph.as_default():
    ##对预测进行四舍五入，因为预测在0-1之间，需要变为0或者1，并且将类型由float32转化为int32
    ##correct_pred是一个布尔型的数据，因为是tf.equal函数的返回值
    correct_pred = tf.equal(tf.cast(tf.round(predictions), tf.int32), labels_)
    ##将correct_pred矩阵转化为float32，再做平均值，得到准确率
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

#一下开始都是固定的运行指令 

##成批的获取数据
        
def get_batches(x, y, batch_size=100):
    n_batches = len(x)//batch_size
    x, y = x[:n_batches*batch_size], y[:n_batches*batch_size]
    for i in range(0, len(x), batch_size):
        yield x[i: i+batch_size], y[i:i+batch_size]
        



In [32]:
##进行RNN训练
epochs = 10

with graph.as_default():
    saver = tf.train.Saver()

with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    iteration = 1
    for e in range(epochs):
        state = sess.run(initial_state)##初始化RNN的层结构
        
        for ii, (x, y) in enumerate(get_batches(train_x, train_y, batch_size),0):
            feed = {inputs_: x,
                    labels_: y[:, None],###因为labels_是一个二维的矩阵，y是一个一维矩阵，所以我们要把y变成二维，用了y[:,None]
                    keep_prob: 0.5,
                    initial_state: state}
            loss, state, _ = sess.run([cost, final_state, optimizer], feed_dict=feed)
            
            ###每5批打印一次迭代次数  批次  损失值
            if iteration%5==0:
                print("Epoch: {}/{}".format(e+1, epochs),
                      "Iteration: {}".format(iteration),
                      "Train loss: {:.3f}".format(loss))
            ###每25批打印一次 验证集的精确度
            if iteration%25==0:
                val_acc = []
                val_state = sess.run(cell.zero_state(batch_size, tf.float32))
                for x, y in get_batches(val_x, val_y, batch_size):
                    feed = {inputs_: x,
                            labels_: y[:, None],
                            keep_prob: 1,
                            initial_state: val_state}
                    batch_acc, val_state = sess.run([accuracy, final_state], feed_dict=feed)
                    val_acc.append(batch_acc)
                print("Val acc: {:.3f}".format(np.mean(val_acc)))
            iteration +=1
    saver.save(sess, "checkpoints/sentiment.ckpt")          

Epoch: 1/10 Iteration: 5 Train loss: 0.245
Epoch: 1/10 Iteration: 10 Train loss: 0.245
Epoch: 1/10 Iteration: 15 Train loss: 0.230
Epoch: 1/10 Iteration: 20 Train loss: 0.208
Epoch: 1/10 Iteration: 25 Train loss: 0.227
Val acc: 0.588
Epoch: 1/10 Iteration: 30 Train loss: 0.239
Epoch: 1/10 Iteration: 35 Train loss: 0.232
Epoch: 1/10 Iteration: 40 Train loss: 0.238
Epoch: 2/10 Iteration: 45 Train loss: 0.211
Epoch: 2/10 Iteration: 50 Train loss: 0.215
Val acc: 0.645
Epoch: 2/10 Iteration: 55 Train loss: 0.182
Epoch: 2/10 Iteration: 60 Train loss: 0.181
Epoch: 2/10 Iteration: 65 Train loss: 0.185
Epoch: 2/10 Iteration: 70 Train loss: 0.181
Epoch: 2/10 Iteration: 75 Train loss: 0.407
Val acc: 0.538
Epoch: 2/10 Iteration: 80 Train loss: 0.232
Epoch: 3/10 Iteration: 85 Train loss: 0.221
Epoch: 3/10 Iteration: 90 Train loss: 0.208
Epoch: 3/10 Iteration: 95 Train loss: 0.171
Epoch: 3/10 Iteration: 100 Train loss: 0.154
Val acc: 0.586
Epoch: 3/10 Iteration: 105 Train loss: 0.421
Epoch: 3/10 Ite

In [33]:
#  测试集测试  Testing
##固定的代码，可以迁移到其他模型上
test_acc = []
with tf.Session(graph=graph) as sess:
    #tf.train.latest_checkpoint  代表获取最后一次生成的
    saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    test_state = sess.run(cell.zero_state(batch_size, tf.float32))
    for ii, (x, y) in enumerate(get_batches(test_x, test_y, batch_size), 0):
        feed = {
            inputs_:x,
            labels_:y[:,None],
            keep_prob:1,
            initial_state: test_state
        }
        batch_acc, test_state = sess.run([accuracy, final_state], feed_dict=feed)
        test_acc.append(batch_acc)
    print('Test accuracy:{:.3f}'.format(np.mean(test_acc)))



INFO:tensorflow:Restoring parameters from checkpoints/sentiment.ckpt
Test accuracy:0.807
